In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [55]:
import numpy as np
import pandas as pd

In [30]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [17]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [7]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [11]:
## reflect an existing database into a new model
# Declaring a Base
Base = automap_base()

## reflect the tables
Base.prepare(engine, reflect=True)

# Save references to classes in data
# Print all the classes mapped to the Base
print(Base.classes.keys())
automap_base().prepare(engine, reflect=True)

# Save reference to classes, 'measurement' and 'station'.
Measurements = Base.classes.measurement
Station = Base.classes.station

['measurement', 'station']


In [ ]:
## We can view all of the classes that automap found

# Refer above for code

In [ ]:
## Save references to each table

# Refer above

In [12]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [40]:
# Inspect columns of measurements and stations
inspector = inspect(engine)
print(inspector.get_table_names())

columns_measurements = inspector.get_columns('measurement')
for column in columns_measurements:
    print(column['name'], column['type'])

['measurement', 'station']
id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


# Exploratory Climate Analysis

In [65]:
# for row in session.query(Measurements, Measurements.station).limit(5).all():
#     print(row)

### Design a query to retrieve the last 12 months of precipitation data and plot the results

## Calculate the date 1 year ago from the last data point in the database
# Retrieve 1st date of data:
print(f'Oldest date in data: {session.query(Measurements.date).order_by(Measurements.date).first()}')
print(f'Most recent date in data: {session.query(Measurements.date).order_by(Measurements.date.desc()).first()}')


# latest_date_measurements = session.query(Measurements.date).order_by(Measurements.date.desc()).first()
# oneyr_priorto_latest = session.query(Measurements.date).order_by(Measurements.date.desc()).first() - dt.timedelta(weeks=52)
# print(oneyr_priorto_latest)

# query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
# print("1 year prior latest date: ", query_date)

# session.query(Measurements.date).filter(Measurements.date > '2016-08-23').order_by(Measurements.date).all()

# following doesn't work
#======================================
# date = dt.datetime(2016, 8, 23)

# results = session.query(Measurements.date, Measurements.prcp).\
#     filter(Measurements.date >= date).all()

# date_prcp = [{'Date': result[1], 'pcrp': result[2]} for result in results]
# date_prcp
#======================================



## Perform a query to retrieve the data and precipitation scores
sel = [Measurements.date, Measurements.prcp]
# Using 22/8, as >= whilst using 2016, 8 23; doesn't include the 23rd data point
date = dt.datetime(2016, 8, 22)

daily_prcp = session.query(*sel).\
    filter(Measurements.date >= date).all()
daily_prcp


## Save the query results as a Pandas DataFrame and set the index to the date column
daily_prcp_df = pd.DataFrame(daily_prcp, columns=["Date", "Precipitation scores"])
new_daily_prcp_df = daily_prcp_df.reset_index(drop=True)
new_daily_prcp_df

## Sort the dataframe by date
#Note already done above, but if it wasn't this is the code that was reset the index.
# print(new_daily_prcp_df.sort_values("Date", ascending=True))


## Use Pandas Plotting with Matplotlib to plot the data
new_daily_prcp_df.set_index('Date', inplace=True)
new_daily_prcp_df.plot.bar()
plt.plot

Oldest date in data: ('2010-01-01',)
Most recent date in data: ('2017-08-23',)


<function matplotlib.pyplot.plot(*args, scalex=True, scaley=True, data=None, **kwargs)>

Error in callback <function flush_figures at 0x000001E068EE14C8> (for post_execute):


KeyboardInterrupt: 

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
